Run the DIVAnd analysis, using DIVAnd cross-validation to estimate the correlation length and epsilon2 input parameters. Use the domain of the observations as the input domain (not the domain of the mask).

In [8]:
using DIVAnd
using CSV
using NCDatasets
using DataFrames
using Statistics
using BenchmarkTools

In [2]:
# To run in command prompt or something
# julia compute_DIVAnd_pmn.jl

var_name = "Oxy"
standard_depth = 0
year = 2010
szn = "OND"
# subsamp_interval = 1

obs_dir = string("C:\\Users\\HourstonH\\Documents\\NEP_climatology\\data\\",
    "value_vs_depth\\14_sep_by_sl_and_year\\")

mask_dir = string("C:\\Users\\HourstonH\\Documents\\NEP_climatology\\data\\",
    "value_vs_depth\\16_diva_analysis\\masks\\")

pmn_dir = string("C:\\Users\\HourstonH\\Documents\\NEP_climatology\\data\\",
    "value_vs_depth\\16_diva_analysis\\pmn\\")

output_dir = string("C:\\Users\\HourstonH\\Documents\\NEP_climatology\\data\\",
    "value_vs_depth\\16_diva_analysis\\")

# years = [1991:1:2020;]
# println(years)
# szns = ["JFM", "AMJ", "JAS", "OND"]

"C:\\Users\\HourstonH\\Documents\\NEP_climatology\\data\\value_vs_depth\\16_diva_analysis\\"

In [3]:
# Open the required files

# Read in standard level data file
obs_filename = string(obs_dir, var_name, "_", standard_depth, "m_", year, 
    "_", szn, ".csv")
    
println(obs_filename)
    
# Pipe operator to dataframe
obs_df = CSV.File(obs_filename) |> DataFrame
    
if size(obs_df)[1] == 0
    println("DataFrame empty -- skipping")
end
    
xobs = obs_df[!, :Longitude]
yobs = obs_df[!, :Latitude]
vobs = obs_df[!, :SL_value]

# Compute anomaly field
vmean = mean(vobs)
vanom = vobs .- vmean
    
# Calculate domain size based on the observations
# Set first guesses for correlation length as 1/10 domain size
domain_size_x_deg = maximum(xobs) - minimum(xobs)
domain_size_y_deg = maximum(yobs) - minimum(xobs)
lenx_guess = domain_size_x_deg/10
leny_guess = domain_size_y_deg/10

println(lenx_guess, " ", leny_guess)

C:\Users\HourstonH\Documents\NEP_climatology\data\value_vs_depth\14_sep_by_sl_and_year\Oxy_0m_2010_OND.csv
1.6538494 19.0285664


In [19]:
# Read in mask
mask_filename = string(mask_dir, var_name, "_", standard_depth, "m_", 
    year, "_", szn, "_mask_6min.nc")
    
mask_ds = Dataset(mask_filename)
    
mask = mask_ds["mask"][:,:]  # [1:subsamp_interval:end, 1:subsamp_interval:end]
mask = Bool.(mask)
# println(typeof(mask))

# Open the pmn netCDF file
pmn_filename = string(pmn_dir, "divand_pmn_for_mask_6min_v3.nc")

pmn_ds = Dataset(pmn_filename)

# Get the inverse of the resolution of the grid
# need to convert from Matrix{Union{Missing, Float64}} to Matrix{Float64}
pm = convert(Array{Float64}, pmn_ds["pm"][:,:])
pn = convert(Array{Float64}, pmn_ds["pn"][:,:])

# Get the 2d mesh grid longitude and latitude
# Could have been made from pmn lon lat or mask lon lat
Lon2d, Lat2d = ndgrid(mask_ds["lon"][:], mask_ds["lat"][:])

close(mask_ds)
close(pmn_ds)

closed NetCDF NCDataset

In [20]:
# Lon2d
println(minimum(pm), " ", maximum(pm))
println(minimum(pn), " ", maximum(pn))
println(typeof(mask))
println(typeof(pm))
println(typeof(Lon2d))

0.0024923218319040544 0.004316465727327514
0.0021583687823886737 0.0021583688291264883
BitMatrix
Matrix{Float64}
Matrix{Float64}


In [6]:
# Set some more parameters

signal_to_noise_ratio = 50.  # Default from Lu ODV session
epsilon2_guess = 1/signal_to_noise_ratio

# Choose number of testing points around the current value of L (corlen)
nl = 1

# Choose number of testing points around the current value of epsilon2
ne = 1

# Choose cross-validation method
# 1: full CV; 2: sampled CV; 3: GCV; 0: automatic choice between the three
method = 3

3

In [ ]:
# Run the cross-validation
bestfactorl, bestfactore, cvval, cvvalues, x2Ddata, y2Ddata, cvinter, xi2D, yi2D = DIVAnd_cv(
    mask, (pm, pn), (Lon2d, Lat2d), (xobs, yobs), vanom, (lenx_guess, leny_guess), 
    epsilon2_guess, nl, ne, method)

In [ ]:
# Apply the correction factors
new_lenx = bestfactorl * lenx_guess
new_leny = bestfactorl * leny_guess
new_epsilon2 = bestfactore * epsilon2_guess

old_params = [lenx_guess, leny_guess, epsilon2_guess]
new_params = [new_lenx, new_leny, new_epsilon2]

for i=[1,2,3]
    println("lenx:", old_params[i], "->", new_params[i])
end

In [ ]:
# Run the analysis
# va = DIVAndrunfi(bool_mask, (pm, pn), (Lon, Lat), (xobs, yobs), vanom,
#                  (new_lenx, new_leny), new_epsilon2)

va = DIVAndrunfi(mask, (pm, pn), (Lon2d, Lat2d), (xobs, yobs), vanom,
                 (lenx_guess, lenx_guess), epsilon2_guess)

# Add the output anomaly back to the mean of the observations
vout = vmean .+ va

In [ ]:
# Export vout as a netCDF file in the same dims as the mask
vout_filename = string(output_dir, var_name, "_", standard_depth, "m_",
    year, "_", szn, "_analysis2d_guess.nc")

vout_ds = Dataset(vout_filename, "c")

# Define lat and lon dims
lon_dim_vout = defDim(vout_ds, "lon", length(vout[:,1]))
lat_dim_vout = defDim(vout_ds, "lat", length(vout[1,:]))

In [ ]:
vout_var = defVar(vout_ds, "vout", Float64, ("lon", "lat"))
vout_var[:,:] = vout

println(vout_ds)

In [ ]:
close(vout_ds)

In [9]:
@btime mean([1,2,3,4,5])

  35.448 ns (1 allocation: 128 bytes)


3.0

In [11]:
lon_subset = Lon2d[1:10, 1:15]
lat_subset = Lat2d[1:10, 1:15]

pm_subset, pn_subset = DIVAnd_metric(lon_subset, lat_subset)

([0.0024923218319040544 0.0024924265561150263 … 0.0024936842772158864 0.0024937891732726776; 0.0024923219353162737 0.002492426607209966 … 0.0024936842759809286 0.0024937892244541664; … ; 0.0024923219353162737 0.002492426607209966 … 0.0024936842759809286 0.0024937892244541664; 0.0024923218319040544 0.0024924265561150263 … 0.0024936842772158864 0.0024937891732726776], [0.0021583688291264883 0.002158368805044173 … 0.002158368805044173 0.00215836878381549; 0.0021583688291264883 0.002158368805044173 … 0.002158368805044173 0.00215836878381549; … ; 0.0021583688291264883 0.002158368805044173 … 0.002158368805044173 0.00215836878381549; 0.0021583688291264883 0.002158368805044173 … 0.002158368805044173 0.00215836878381549])

In [13]:
pm_subset

typeof(pm_subset)

Matrix{Float64} (alias for Array{Float64, 2})

In [14]:
pm_sub2 = pm[1:10, 1:15]
pn_sub2 = pn[1:10, 1:15]

10×15 Matrix{Union{Missing, Float64}}:
 0.00215837  0.00215837  0.00215837  …  0.00215837  0.00215837  0.00215837
 0.00215837  0.00215837  0.00215837     0.00215837  0.00215837  0.00215837
 0.00215837  0.00215837  0.00215837     0.00215837  0.00215837  0.00215837
 0.00215837  0.00215837  0.00215837     0.00215837  0.00215837  0.00215837
 0.00215837  0.00215837  0.00215837     0.00215837  0.00215837  0.00215837
 0.00215837  0.00215837  0.00215837  …  0.00215837  0.00215837  0.00215837
 0.00215837  0.00215837  0.00215837     0.00215837  0.00215837  0.00215837
 0.00215837  0.00215837  0.00215837     0.00215837  0.00215837  0.00215837
 0.00215837  0.00215837  0.00215837     0.00215837  0.00215837  0.00215837
 0.00215837  0.00215837  0.00215837     0.00215837  0.00215837  0.00215837

In [15]:
ismissing(pm_sub2)

false

In [18]:
convert(Array{Float64}, pm_sub2)

10×15 Matrix{Float64}:
 0.00249232  0.00249243  0.00249253  …  0.00249358  0.00249368  0.00249379
 0.00249232  0.00249243  0.00249253     0.00249358  0.00249368  0.00249379
 0.00249232  0.00249243  0.00249253     0.00249358  0.00249368  0.00249379
 0.00249232  0.00249243  0.00249253     0.00249358  0.00249368  0.00249379
 0.00249232  0.00249243  0.00249253     0.00249358  0.00249368  0.00249379
 0.00249232  0.00249243  0.00249253  …  0.00249358  0.00249368  0.00249379
 0.00249232  0.00249243  0.00249253     0.00249358  0.00249368  0.00249379
 0.00249232  0.00249243  0.00249253     0.00249358  0.00249368  0.00249379
 0.00249232  0.00249243  0.00249253     0.00249358  0.00249368  0.00249379
 0.00249232  0.00249243  0.00249253     0.00249358  0.00249368  0.00249379